In [0]:
#default_exp mem

In [0]:
#hide
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#hide
!pip install nbdev
!pip install fastcore

In [0]:
#hide
%cd /content/drive/My\ Drive/mx_utils

In [0]:
#hide
#test_flag_imports
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

# Memory management.

> Code snippets to help memory management in Google Colab.


## GPU

Use `!nvidia-smi` to get GPU details and current memory usage.

`torch.cuda.empty_cache()` clears GPU cache to free up GPU memory

## CPU RAM

In [0]:
#exports
def show_ram_usage():
  import psutil
  "Check overall RAM usage" 
  pu_mem = psutil.Process(os.getpid())
  print('RAM usage: {} GB'.format(pu_mem_,memory_info([0]/1024 ** 3)))

In [0]:
#exports
def purge_mem(vars=None, mem=True):
  "Delete `vars` and recycle CPU memory. Show new RAM if `mem=True`"
  for var in vars:
    del var
  gc.collect()
  if mem: show_ram_usage()

## Dataframe memory usage

In [0]:
#exports
def show_df_mem_usage(df):
  "Check dataframe `df` memory use"
  print('Shape: ', df.shape)
  print('Total Mem usage: ', df.memory_usage().sum())
  df.memory_usage()

In [0]:
#exports
def reduce_df_mem_usage(df, verbose=True):
  "Reduce size of dataframe `df` columns to minimise memory usage"
  numerics = ['int16', 'int32', 'int64', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2    
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)  
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)    
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df